In [2]:
import numpy as np
import datetime
import cv2 
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Activation,Conv2D,Dense,MaxPool2D,Dropout,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
from IPython.display import Image, display

from tensorflow.keras.applications import InceptionV3,Xception,EfficientNetB3
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Model
from keras.layers import Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2

Base_directory = '/kaggle/input/flamedatasetfireclassification'
test_path = 'Test/Test'
Training_path = 'Training/Training'
input_shape = (254,254,3)
batch = 32
labels = ['Fire','No_Fire']
Full_Training_path = '{0}/{1}'.format(Base_directory,Training_path)
Full_Test_path = '{0}/{1}'.format(Base_directory,test_path)

train_images = ImageDataGenerator(rotation_range=45,
                                # width_shift_range=0.1,
                                 #height_shift_range=0.1,
                                 horizontal_flip=True,
                                 vertical_flip=True,
                                 rescale=1.0/255,
                                 zoom_range=0.4,
                                 shear_range=0.2,
                                 fill_mode='nearest',
                                 validation_split=0.2                                           
#            preprocessing_function = contrast_stretching,
#             preprocessing_function = HE,
                                #preprocessing_function = AHE
                             )

train_generator = train_images.flow_from_directory(Full_Training_path, 
                                               target_size=(254,254),
                                               color_mode='rgb',
                                               class_mode='binary',
                                               batch_size=batch,
                                               shuffle=True,
                                               subset='training')

validation_generator = train_images.flow_from_directory(Full_Training_path, 
                                                    target_size=(254,254),
                                                    color_mode='rgb',
                                                    class_mode='binary',
                                                    batch_size=batch,
                                                    shuffle=True,
                                                    subset='validation')

test_images = ImageDataGenerator(rescale=1.0/255)

test_generator = test_images.flow_from_directory(Full_Test_path, 
                                                target_size=(254,254), 
                                                color_mode='rgb', 
                                                class_mode='binary',
                                                shuffle=False,
                                                batch_size=batch)
    


Found 31501 images belonging to 2 classes.
Found 7874 images belonging to 2 classes.
Found 8617 images belonging to 2 classes.


In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 252, 252, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 252, 252, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 124, 124, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        1

In [4]:
my_callbacks = [
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1),
    ModelCheckpoint(filepath='Xceptionnet_1104_{epoch:02d}_{val_accuracy:.04f}.h5',
                    save_best_only=False)]

history = model.fit(train_generator,
            epochs=50,
            validation_data = test_generator,            
            callbacks=my_callbacks,
            verbose=1    )

Epoch 1/50
985/985 [==============================] - 774s 782ms/step - loss: 0.4172 - accuracy: 0.8454 - val_loss: 1.0715 - val_accuracy: 0.5996
Epoch 2/50
985/985 [==============================] - 480s 487ms/step - loss: 0.2280 - accuracy: 0.9240 - val_loss: 1.3613 - val_accuracy: 0.6094
Epoch 3/50
985/985 [==============================] - 471s 478ms/step - loss: 0.2271 - accuracy: 0.9354 - val_loss: 1.1945 - val_accuracy: 0.6243
Epoch 4/50
985/985 [==============================] - 479s 486ms/step - loss: 0.1800 - accuracy: 0.9468 - val_loss: 0.9610 - val_accuracy: 0.6553
Epoch 5/50
985/985 [==============================] - 473s 480ms/step - loss: 0.3864 - accuracy: 0.9472 - val_loss: 1.9874 - val_accuracy: 0.6043
Epoch 6/50
985/985 [==============================] - 465s 472ms/step - loss: 0.2272 - accuracy: 0.9511 - val_loss: 3.8328 - val_accuracy: 0.6862
Epoch 7/50
985/985 [==============================] - 459s 466ms/step - loss: 0.3003 - accuracy: 0.9532 - val_loss: 14.9388 